### Experimenting with news feeds to create data for the chat engine

In [49]:

from urllib.request import urlopen
from bs4 import BeautifulSoup
import xmltodict, json
import html
from html.parser import HTMLParser


In [ ]:

def readpage(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text

def readfeed(url):

    t = xmltodict.parse(urlopen(url).read())
    return t

def get_headlines():
    urls = {
        "NY Times": "http://rss.nytimes.com/services/xml/rss/nyt/HomePage.xml",
        "CNN": "http://rss.cnn.com/rss/cnn_latest.rss"
        }
    output = ''
    for v in urls:
        output += (f"\n\nTop Headlines from {v}:\n")
        jtext = readfeed(urls[v])
        h = html.parser
        headline_count = 0
        for i in range(len(jtext['rss']['channel']['item'])):
            if (headline_count<3):
                title = h.unescape(jtext['rss']['channel']['item'][i]['title'])
                description = h.unescape(jtext['rss']['channel']['item'][i]['description'])
                # print(title,":\n",description)
                output += '\n'+title+":\n"+description
                # print(json.dumps(jtext['rss']['channel']['item'][i],indent=4))
                headline_count +=1
    return output

if "__name__" == "__main__":
    print(get_headlines())